# Testing lightcurve plots

## Downloading data from ATLAS API

In [10]:
#!/usr/bin/env python3
# type: ignore
import os
import re
import sys
import time
from io import StringIO
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
import requests

# Define the input/output directory
input_dir_txt = r'c:/Users/jhzhe/Documents/MPhys_Neutrino/input_data/txt_data'
input_dir_csv = r'c:/Users/jhzhe/Documents/MPhys_Neutrino/input_data/csv_data'
output_dir = r'c:/Users/jhzhe/Documents/MPhys_Neutrino/output_data'

# Ensure the directory exists
os.makedirs(output_dir, exist_ok=True)


BASEURL = "https://fallingstar-data.com/forcedphot"
# BASEURL = "http://127.0.0.1:8000"

if os.environ.get("ATLASFORCED_SECRET_KEY"):
    token = os.environ.get("ATLASFORCED_SECRET_KEY")
    print("Using stored token")
else:
    data = {"username": "JinZheng", "password": "NB,E7,SNCL5mx"}

    resp = requests.post(url=f"{BASEURL}/api-token-auth/", data=data)

    if resp.status_code == 200:
        token = resp.json()["token"]
        print(f"Your token is {token}")
        print("Store this by running/adding to your .zshrc file:")
        print(f'export ATLASFORCED_SECRET_KEY="{token}"')
    else:
        print(f"ERROR {resp.status_code}")
        print(resp.text)
        sys.exit()


headers = {"Authorization": f"Token {token}", "Accept": "application/json"}

Your token is 938191f3499bfce3ac5b3566e298b2c9106d84d3
Store this by running/adding to your .zshrc file:
export ATLASFORCED_SECRET_KEY="938191f3499bfce3ac5b3566e298b2c9106d84d3"


In [11]:

def fetch_data(ra, dec, mjd_min=58582.8, mjd_max=58782.8, index=1):
    task_url = None
    while not task_url:
        with requests.Session() as s:
            # alternative to token auth
            # s.auth = ('USERNAME', 'PASSWORD')
            resp = s.post(f"{BASEURL}/queue/", headers=headers, data={"ra": ra, "dec": dec, "mjd_min": mjd_min,
                                                                      "mjd_max": mjd_max})

            if resp.status_code == 201:  # successfully queued
                task_url = resp.json()["url"]
                print(f"The task URL is {task_url}")
            elif resp.status_code == 429:  # throttled
                message = resp.json()["detail"]
                print(f"{resp.status_code} {message}")
                t_sec = re.findall(r"available in (\d+) seconds", message)
                t_min = re.findall(r"available in (\d+) minutes", message)
                if t_sec:
                    waittime = int(t_sec[0])
                elif t_min:
                    waittime = int(t_min[0]) * 60
                else:
                    waittime = 10
                print(f"Waiting {waittime} seconds")
                time.sleep(waittime)
            else:
                print(f"ERROR {resp.status_code}")
                print(resp.text)
                sys.exit()


    result_url = None
    taskstarted_printed = False
    while not result_url:
        with requests.Session() as s:
            resp = s.get(task_url, headers=headers)

            if resp.status_code == 200:  # HTTP OK
                if resp.json()["finishtimestamp"]:
                    result_url = resp.json()["result_url"]
                    print(f"Task is complete with results available at {result_url}")
                elif resp.json()["starttimestamp"]:
                    if not taskstarted_printed:
                        print(f"Task is running (started at {resp.json()['starttimestamp']})")
                        taskstarted_printed = True
                    time.sleep(2)
                else:
                    print(f"Waiting for job to start (queued at {resp.json()['timestamp']})")
                    time.sleep(4)
            else:
                print(f"ERROR {resp.status_code}")
                print(resp.text)
                sys.exit()

    with requests.Session() as s:
        textdata = s.get(result_url, headers=headers).text
        file_path_data_txt = os.path.join(input_dir_txt, f'text_data{index}.txt')
        with open(file_path_data_txt, 'w') as f:
            f.write(textdata)
        print(f"Raw text data saved to {file_path_data_txt}")
        # if we'll be making a lot of requests, keep the web queue from being
        # cluttered (and reduce server storage usage) by sending a delete operation
        # s.delete(task_url, headers=headers).json()

    dfresult = pd.read_csv(StringIO(textdata.replace("###", "")), delim_whitespace=True)

    file_path_data = os.path.join(input_dir_csv, f'data{index}.csv')
    
    dfresult.to_csv(file_path_data)
    """
    file_path_image = os.path.join(output_dir, f'plot_data{index}.png')
    fig = plt.figure()
    plt.plot(dfresult['MJD'], dfresult['uJy'])
    plt.savefig(file_path_image)
    plt.close(fig)
    """

## Fetching/Plotting data for AT2019dsg RA/DEC

In [8]:
import random
from plot_atlas_fp import plotter
from clean import CleanLoop
from PIL import Image
import logging
import os

logging.basicConfig(level=logging.INFO) 
log = logging.getLogger(__name__)

In [4]:
# Fetching data into 2 formats, csv and txt

ra_min = 313.76
ra_max = 314.76
dec_min = 13.7
dec_max = 14.7

ra_list = []
dec_list = []

for i in range(50):
    ra_list.append(round(random.uniform(ra_min, ra_max), 2))
    dec_list.append(round(random.uniform(dec_min, dec_max),2))

#for i, (ra, dec) in enumerate(zip(ra_list, dec_list)):
 #   fetch_data(ra, dec, index=i)

In [5]:
# Initiliasing the plotter class
resultsFilePath = [os.path.join(input_dir_txt, file) for file in os.listdir(input_dir_txt) if file.endswith('.txt')]

myPlotter = plotter(
    log = log,
    resultFilePaths = resultsFilePath,
    outputDirectory = output_dir,
    plotType = "png",
    RA_list = ra_list,
    dec_list = dec_list
)

In [6]:
# Plotting data and saving to output_dir
# to set RA/dec for each plot, need to set title in single_plot_result method
# will need to pass RA/dec list, to count where we are in list, set var, coord_index = 0 at start of class, increment counter after each plot

plotPaths = myPlotter.plot_compile()

INFO:__main__:starting the ``plot_compile`` method
INFO:__main__:starting the ``plot_single_result_compile_o`` method
INFO:__main__:starting the ``read_and_sigma_clip_data`` function
INFO:__main__:completed ``plot_single_result_compile_c``
INFO:__main__:starting the ``plot_single_result_compile_o`` method
INFO:__main__:starting the ``read_and_sigma_clip_data`` function
INFO:__main__:completed ``plot_single_result_compile_o``
INFO:__main__:starting the ``plot_single_result_compile_o`` method
INFO:__main__:starting the ``read_and_sigma_clip_data`` function
INFO:__main__:completed ``plot_single_result_compile_c``
INFO:__main__:starting the ``plot_single_result_compile_o`` method
INFO:__main__:starting the ``read_and_sigma_clip_data`` function
INFO:__main__:completed ``plot_single_result_compile_o``
INFO:__main__:starting the ``plot_single_result_compile_o`` method
INFO:__main__:starting the ``read_and_sigma_clip_data`` function
INFO:__main__:completed ``plot_single_result_compile_c``
INFO

In [28]:
# compiling plots Note: need to display RA/dec for each plot

plot_dict = {}
plotsFilePath = [os.path.join(output_dir, file) for file in os.listdir(output_dir) if file.endswith('.png')]

for i in range(len(resultsFilePath)):
    plot_dict['plot'+str(i)] = Image.open(plotsFilePath[i])

compiled_list = [x for x in plot_dict.values()]

plot_dict['plot1'].save(r"C:\Users\jhzhe\Documents\MPhys_Neutrino\output_data\compiled.pdf", save_all=True, append_images = compiled_list)

for i in range(len(plot_dict)):
    plot_dict['plot'+str(i)].close()

In [24]:
# removes any files with png post compiling
for file in os.listdir(output_dir):
    if file.endswith('.png'):
        file_path = os.path.join(output_dir, file)
        try:
            os.remove(file_path)
        except PermissionError:
            # If the file is in use, wait a moment and try again
            time.sleep(0.5)
            os.remove(file_path)

In [ ]:
def plot_compiler(output_dir: str):
    '''
    Compiles all plot.pngs found in output directory and compiles them into a single pdf, the pngs are then deleted

    Arguments:
    output_dir: output directory
    
    Outputs:
    None
    '''
    plot_dict = {}
    plotsFilePath = [os.path.join(output_dir, file) for file in os.listdir(output_dir) if file.endswith('.png')]

    for i in range(len(plotsFilePath)):
        plot_dict['plot'+str(i)] = Image.open(plotsFilePath[i])

    compiled_list = [x for x in plot_dict.values()]

    plot_dict['plot1'].save(r"C:\Users\jhzhe\Documents\MPhys_Neutrino\output_data\compiled.pdf", save_all=True, append_images = compiled_list)

    for i in range(len(plot_dict)):
        plot_dict['plot'+str(i)].close()

    for file in os.listdir(output_dir):
        if file.endswith('.png'):
            file_path = os.path.join(output_dir, file)
            try:
                os.remove(file_path)
            except PermissionError:
                # If the file is in use, wait a moment and try again
                time.sleep(0.5)
                os.remove(file_path)
    
    print('Plots compiled, found in '+ str(output_dir))

## Testing Cleaning with Sofia

In [2]:
import configparser
from download import Credentials

def load_config(config_file):
    config = configparser.ConfigParser()
    config.read(config_file)
    return config


In [4]:
config = load_config(r"C:\Users\jhzhe\ATCleanRepo\config.ini")

credentials = Credentials(
    config["credentials"]["atlas_username"],
    config["credentials"]["atlas_password"],
    config["credentials"]["tns_api_key"],
    config["credentials"]["tns_id"],
    config["credentials"]["tns_bot_name"]
)


TypeError: Credentials.__init__() missing 3 required positional arguments: 'tns_api_key', 'tns_id', and 'tns_bot_name'